# k-Nearest Neighbors

The model for kNN is the entire training dataset. When a prediction is required for a unseen data instance, the kNN algorithm will search through the training dataset for the k-most similar instances. The prediction attribute of the most similar instances is summarized and returned as the prediction for the unseen instance.

The similarity measure is dependent on the type of data. For real-valued data, the Euclidean distance can be used. Other other types of data such as categorical or binary data, Hamming distance can be used.

In the case of regression problems, the average of the predicted attribute may be returned. In the case of classification, the most prevalent class may be returned.

## Exercise 1 - Explore the Data

The test problem we will be using is iris classification. The problem is comprised of 150 observations of iris flowers from three different species. There are 4 measurements of given flowers: sepal length, sepal width, petal length and petal width, all in the same unit of centimeters. The predicted attribute is the species, which is one of setosa, versicolor or virginica.

It is a standard dataset where the species is known for all instances. As such we can split the data into training and test datasets and use the results to evaluate our algorithm implementation. Good classification accuracy on this problem is above 90% correct, typically 96% or better.

You can download the dataset for free from [iris.data](https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data), see the resources section for further details.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

iris = pd.read_csv('iris.data', header=None)
iris.columns = ['sep_len', 'sep_wid', 'pet_len', 'pet_wid', 'class']
iris.head()

,sep_len,sep_wid,pet_len,pet_wid,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [2]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(iris.values, test_size=0.4)

## Exercise 2 - Build a k-Nearest Neighbors Class

The derivation can be [found here on Wikipedia](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm). Here is a useful link that explains how to do a [weighted KNN](http://www.csee.umbc.edu/~tinoosh/cmpe650/slides/K_Nearest_Neighbor_Algorithm.pdf).

The general steps are:
- Locate the k most similar data instances from a test instance
    - This is done by calculating the distance from each instance in the data set to the test instance
- Return the predicted class label 
- Return an accuracy measurement

In [20]:
# generate all steps and convert to class
# 1) EuclideanDistance

import math
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)


In [21]:
# test data
data1 = [2, 2, 2, 'a']
data2 = [4, 4, 4, 'b']
distance = euclideanDistance(data1, data2, 3)
print('Distance: ' + repr(distance))

Distance: 3.4641016151377544


In [39]:
# 2) get the nearest neighbors
import operator

def getNeighbors(trainSet, testInstance, k):
        distances = []
        length = len(testInstance) - 1
        
        for x in range(len(trainSet)):
            dist = euclideanDistance(testInstance, trainSet[x], length)
            distances.append((trainSet[x], dist))
        
        # sort on the distance
        distances.sort(key=itemgetter(1))
    
        neighbors = []
        for x in range(k):
            neighbors.append(distances[x][0])
        
        return neighbors



In [40]:
# test nearest neighbors
trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5]
k = 1
neighbors = getNeighbors(trainSet, testInstance, 1)
print(neighbors)

[[4, 4, 4, 'b']]


In [37]:
# 3) getting majority voted response from neighbors. Class is last attribute.
def getResponse(neighbors):
        classVotes = {}
        
        for x in range(len(neighbors)):
            response = neighbors[x][-1]
            
            if response in classVotes:
                classVotes[response] += 1
            else:
                classVotes[response] = 1
                
        sortedVotes = sorted(classVotes.items(), key=itemgetter(1), reverse=True)
        
        return sortedVotes[0][0]

In [38]:
# test response
neighbors = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
response = getResponse(neighbors)
print(response)

a


In [41]:
# 4) Accuracy

def getAccuracy(testSet, predictions):
        correct = 0
        
        for x in range(len(testSet)):
            if testSet[x][-1] is predictions[x]:
                correct += 1
                
        return (correct / float(len(testSet)))

In [42]:
# test accuracy
testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
predictions = ['a', 'a', 'a']
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

0.6666666666666666


In [18]:
import math
from operator import itemgetter

class KNN():
    def __init__(self):
        pass
    
    # obtain the distance between two arrays of numbers
    def euclideanDistance(self, instance1, instance2, length):
        distance = 0
        
        for x in range(length):
            distance +=  math.pow((instance1[x] - instance2[x]), 2)

        return math.sqrt(distance)
    
    def getNeighbors(self, trainSet, testInstance, k):
        distances = []
        length = len(testInstance) - 1
        
        for x in range(len(trainSet)):
            dist = self.euclideanDistance(testInstance, trainSet[x], length)
            distances.append((trainSet[x], dist))
        
        distances.sort(key=itemgetter(1))
    
        neighbors = []
        for x in range(k):
            neighbors.append(distances[x][0])
        
        return neighbors
    
    def getResponse(self, neighbors):
        classVotes = {}
        
        for x in range(len(neighbors)):
            response = neighbors[x][-1]
            
            if response in classVotes:
                classVotes[response] += 1
            else:
                classVotes[response] = 1
                
        sortedVotes = sorted(classVotes.items(), key=itemgetter(1), reverse=True)
        
        return sortedVotes[0][0]
    
    def getAccuracy(self, testSet, predictions):
        correct = 0
        
        for x in range(len(testSet)):
            if testSet[x][-1] is predictions[x]:
                correct += 1
                
        return (correct / float(len(testSet)))
    
    def predict(self, trainSet, testSet, k=3):
        prediction = []
        
        for x in range(len(testSet)):
            neighbors = self.getNeighbors(trainSet, testSet[x], k)
            result = self.getResponse(neighbors)
            prediction.append(result)
            print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
            
        accuracy = self.getAccuracy(testSet, prediction)
        print('Accuracy: ' + repr(accuracy) + '%')
            

## Exercise 3 - Try it out on the Iris Data Set. 

In [19]:
knn = KNN()
knn.predict(train, test, 3)

> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-virginica', actual='Iris-virginica'
> predicted='Iris-versicolor', actual='Iris-versicolor'
> predicted='Iris-versicolor', actual='Iris-versicolor'
> predicted='Iris-virginica', actual='Iris-virginica'
> predicted='Iris-virginica', actual='Iris-virginica'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-virginica', actual='Iris-virginica'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-virginica', actual='Iris-virginica'
> predicted='Iris-versicolor', actual='Iris-versicolor'
> predicted='Iris-versicolor', actual='Iris-versicolor'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-versicolor', actual='Iris-virginica'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-v

## Exercise 4 - Check via scikit-learn. Plot the decision regions.

In [50]:
from sklearn import neighbors
X = iris.iloc[:, 0:4]
y = iris['class']

clf = neighbors.KNeighborsClassifier(n_neighbors=3)
clf.fit(X, y)
Z = clf.predict(X)
accuracy = clf.score(X, y)
print("Predicted model accuracy: " + str(accuracy))

Predicted model accuracy: 0.96


In [51]:
iris['Z'] = Z
iris.iloc[:,4:6]

,class,Z
0,Iris-setosa,Iris-setosa
1,Iris-setosa,Iris-setosa
2,Iris-setosa,Iris-setosa
3,Iris-setosa,Iris-setosa
4,Iris-setosa,Iris-setosa
5,Iris-setosa,Iris-setosa
6,Iris-setosa,Iris-setosa
7,Iris-setosa,Iris-setosa
8,Iris-setosa,Iris-setosa
9,Iris-setosa,Iris-setosa
